In [7]:
# 패션 MNIST - 훈련세트, 테스트 세트 
from tensorflow import keras

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

In [8]:
# 모델 생성 함수 - 층 추가가 가능 
def model_fn(a_layer = None):
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(28,28))) # 입력층
    model.add(keras.layers.Flatten()) 
    model.add(keras.layers.Dense(100, activation='relu'))
    if a_layer:
        model.add(a_layer)
    
    model.add(keras.layers.Dense(10, activation='softmax'))

    return model

In [9]:
model = model_fn(keras.layers.Dropout(0.3))
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)                  │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 100)                 │          78,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 10)                  │           1,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 79,510 (310.59 KB)

 Trainable params: 79,510 (310.59 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
train_scaled = train_input / 255
test_scaled = test_input / 255
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 검증 세트 정확도가 떨어지기 직전 시점에 모델을 파일로 저장
# 과대 적합 직전 시점에 모델을 파일로 저장!
checkpoint_cb = keras.callbacks.ModelCheckpoint("best-model.keras", save_best_only=True) 

# 과대 적합 되기 직전 까지만 훈련하고 종료 - 조기종료
# 검증 세트의 정확도가 2회 이상 올라가지 않으면 조기 종료(patience)
# restore_best_weights : 가장 최적의 가중치로 복구 
early_stopping_cb = keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)
model.fit(train_scaled, train_target, epochs=20, validation_data=(test_scaled, test_target), callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7462 - loss: 0.7269 - val_accuracy: 0.8404 - val_loss: 0.4511
Epoch 2/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8426 - loss: 0.4360 - val_accuracy: 0.8575 - val_loss: 0.4055
Epoch 3/20
1778/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8575 - loss: 0.3971

In [ ]:
model = keras.models.load_model("best-model.keras")

In [ ]:
model.evaluate(test_scaled, test_target)